# Keras模型训练

In [34]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split


print('TensorFlow Version:', tf.__version__)
for device in tf.config.list_physical_devices('GPU'):
    print(device)

TensorFlow Version: 2.2.0
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')


In [32]:
def generate_model():
    inputs = tf.keras.Input(shape=(4,))
    outputs = tf.keras.layers.Dense(32, activation='relu')(inputs)
    outputs = tf.keras.layers.Dense(3)(outputs)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-2),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
    )
    
    return model

iris = load_iris()
x, y = iris.data, iris.target
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

## 基本模型训练

In [33]:
print('Model Trainning:')
model = generate_model()
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2)
print()

print('Evaluate in Test:')
result = model.evaluate(x_test, y_test, batch_size=32)
print('Test Loss, Test Acc:', result)
print()

print('Generate Predictions:')
predictions = model.predict(x_test)
print(predictions.shape)

Model Trainning:
Epoch 1/10
4/4 [==============================] - 0s 16ms/step - loss: 1.4288 - sparse_categorical_accuracy: 0.3056 - val_loss: 0.8710 - val_sparse_categorical_accuracy: 0.7037
Epoch 2/10
4/4 [==============================] - 0s 5ms/step - loss: 0.8696 - sparse_categorical_accuracy: 0.6944 - val_loss: 1.1810 - val_sparse_categorical_accuracy: 0.4815
Epoch 3/10
4/4 [==============================] - 0s 6ms/step - loss: 0.8543 - sparse_categorical_accuracy: 0.7222 - val_loss: 0.9607 - val_sparse_categorical_accuracy: 0.4815
Epoch 4/10
4/4 [==============================] - 0s 5ms/step - loss: 0.7218 - sparse_categorical_accuracy: 0.7500 - val_loss: 0.7087 - val_sparse_categorical_accuracy: 0.7037
Epoch 5/10
4/4 [==============================] - 0s 5ms/step - loss: 0.6357 - sparse_categorical_accuracy: 0.6852 - val_loss: 0.6372 - val_sparse_categorical_accuracy: 0.7037
Epoch 6/10
4/4 [==============================] - 0s 6ms/step - loss: 0.5839 - sparse_categorical_accu

## 类别加权与样本加权

In [38]:
class_weight = {0: 0.8, 1: 1.0, 2: 1.2}
sample_weight = np.ones(len(x_train))
sample_weight[5] = sample_weight[10] = 2.0

print('Model Trainning:')
model = generate_model()
model.fit(x_train, y_train, 
          epochs=10, 
          batch_size=32, 
          validation_split=0.2,
          sample_weight=sample_weight,
          class_weight=class_weight)
print()

Model Trainning:
Epoch 1/10
4/4 [==============================] - 0s 17ms/step - loss: 1.3252 - sparse_categorical_accuracy: 0.3611 - val_loss: 0.9403 - val_sparse_categorical_accuracy: 0.3704
Epoch 2/10
4/4 [==============================] - 0s 6ms/step - loss: 0.9277 - sparse_categorical_accuracy: 0.5741 - val_loss: 0.7819 - val_sparse_categorical_accuracy: 0.7037
Epoch 3/10
4/4 [==============================] - 0s 6ms/step - loss: 0.8214 - sparse_categorical_accuracy: 0.6574 - val_loss: 0.6213 - val_sparse_categorical_accuracy: 0.7037
Epoch 4/10
4/4 [==============================] - 0s 6ms/step - loss: 0.6128 - sparse_categorical_accuracy: 0.7870 - val_loss: 0.7243 - val_sparse_categorical_accuracy: 0.4815
Epoch 5/10
4/4 [==============================] - 0s 6ms/step - loss: 0.5769 - sparse_categorical_accuracy: 0.7222 - val_loss: 0.5428 - val_sparse_categorical_accuracy: 0.9630
Epoch 6/10
4/4 [==============================] - 0s 6ms/step - loss: 0.4862 - sparse_categorical_accu

## 回调函数

### EarlyStopping

In [44]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_sparse_categorical_accuracy',
        min_delta=1e-2,
        patience=5,
        verbose=1
    )
]

print('Model Trainning:')
model = generate_model()
model.fit(x_train, y_train, 
          epochs=100, 
          batch_size=32, 
          callbacks=callbacks,
          validation_split=0.2)
print()

Model Trainning:
Epoch 1/100
4/4 [==============================] - 0s 16ms/step - loss: 1.9201 - sparse_categorical_accuracy: 0.2870 - val_loss: 1.0141 - val_sparse_categorical_accuracy: 0.4444
Epoch 2/100
4/4 [==============================] - 0s 6ms/step - loss: 1.1723 - sparse_categorical_accuracy: 0.3241 - val_loss: 0.8723 - val_sparse_categorical_accuracy: 0.4815
Epoch 3/100
4/4 [==============================] - 0s 5ms/step - loss: 0.9350 - sparse_categorical_accuracy: 0.5370 - val_loss: 1.0209 - val_sparse_categorical_accuracy: 0.4815
Epoch 4/100
4/4 [==============================] - 0s 5ms/step - loss: 0.8304 - sparse_categorical_accuracy: 0.6111 - val_loss: 0.9377 - val_sparse_categorical_accuracy: 0.3333
Epoch 5/100
4/4 [==============================] - 0s 5ms/step - loss: 0.7311 - sparse_categorical_accuracy: 0.6019 - val_loss: 0.6729 - val_sparse_categorical_accuracy: 0.7778
Epoch 6/100
4/4 [==============================] - 0s 5ms/step - loss: 0.6127 - sparse_categorica

### 动态学习率与模型保存

In [46]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath='./checkpoints/keras_model_{epoch}',
        save_best_only=True,
        monitor='val_loss',
        save_weights_only=True,
        verbose=1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_sparse_categorical_accuracy',
        mode='max',
        factor=0.5,
        patience=3,
        verbose=1
    )
]

print('Model Trainning:')
model = generate_model()
model.fit(x_train, y_train, 
          epochs=50, 
          batch_size=32, 
          callbacks=callbacks,
          validation_split=0.2)
print()

Model Trainning:
Epoch 1/50
1/4 [======>.......................] - ETA: 0s - loss: 1.6634 - sparse_categorical_accuracy: 0.4062
Epoch 00001: val_loss improved from inf to 1.04624, saving model to ./checkpoints/keras_model_1
4/4 [==============================] - 0s 18ms/step - loss: 1.4805 - sparse_categorical_accuracy: 0.3333 - val_loss: 1.0462 - val_sparse_categorical_accuracy: 0.2963 - lr: 0.0100
Epoch 2/50
1/4 [======>.......................] - ETA: 0s - loss: 1.1057 - sparse_categorical_accuracy: 0.4375
Epoch 00002: val_loss improved from 1.04624 to 0.88813, saving model to ./checkpoints/keras_model_2
4/4 [==============================] - 0s 7ms/step - loss: 1.1197 - sparse_categorical_accuracy: 0.3148 - val_loss: 0.8881 - val_sparse_categorical_accuracy: 0.5185 - lr: 0.0100
Epoch 3/50
1/4 [======>.......................] - ETA: 0s - loss: 1.0457 - sparse_categorical_accuracy: 0.2188
Epoch 00003: val_loss improved from 0.88813 to 0.84679, saving model to ./checkpoints/keras_model

Epoch 22/50
1/4 [======>.......................] - ETA: 0s - loss: 0.3625 - sparse_categorical_accuracy: 1.0000
Epoch 00022: val_loss did not improve from 0.47429
4/4 [==============================] - 0s 6ms/step - loss: 0.3919 - sparse_categorical_accuracy: 0.9907 - val_loss: 0.4758 - val_sparse_categorical_accuracy: 0.9630 - lr: 3.1250e-04
Epoch 23/50
1/4 [======>.......................] - ETA: 0s - loss: 0.4101 - sparse_categorical_accuracy: 0.9688
Epoch 00023: val_loss did not improve from 0.47429
4/4 [==============================] - 0s 5ms/step - loss: 0.3912 - sparse_categorical_accuracy: 0.9907 - val_loss: 0.4753 - val_sparse_categorical_accuracy: 0.9630 - lr: 3.1250e-04
Epoch 24/50
1/4 [======>.......................] - ETA: 0s - loss: 0.3905 - sparse_categorical_accuracy: 1.0000
Epoch 00024: val_loss did not improve from 0.47429

Epoch 00024: ReduceLROnPlateau reducing learning rate to 0.00015624999650754035.
4/4 [==============================] - 0s 5ms/step - loss: 0.3906

1/4 [======>.......................] - ETA: 0s - loss: 0.3689 - sparse_categorical_accuracy: 1.0000
Epoch 00043: val_loss did not improve from 0.47358
4/4 [==============================] - 0s 5ms/step - loss: 0.3881 - sparse_categorical_accuracy: 0.9907 - val_loss: 0.4738 - val_sparse_categorical_accuracy: 0.9630 - lr: 2.4414e-06
Epoch 44/50
1/4 [======>.......................] - ETA: 0s - loss: 0.4590 - sparse_categorical_accuracy: 1.0000
Epoch 00044: val_loss did not improve from 0.47358
4/4 [==============================] - 0s 5ms/step - loss: 0.3881 - sparse_categorical_accuracy: 0.9907 - val_loss: 0.4738 - val_sparse_categorical_accuracy: 0.9630 - lr: 2.4414e-06
Epoch 45/50
1/4 [======>.......................] - ETA: 0s - loss: 0.3826 - sparse_categorical_accuracy: 1.0000
Epoch 00045: val_loss did not improve from 0.47358

Epoch 00045: ReduceLROnPlateau reducing learning rate to 1.220703097715159e-06.
4/4 [==============================] - 0s 5ms/step - loss: 0.3881 - sparse_cat